In [ ]:
!pip install datasets transformers

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import tensorflow as tf

import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
df = pd.read_csv("/kaggle/input/mpst-movie-plot-synopses-with-tags/mpst_full_data.csv")
df.columns

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
train = df.loc[df["split"]=="train"]
test = df.loc[df["split"]=="test"]
val = df.loc[df["split"]=="val"]


In [ ]:
train.tags[0]

In [ ]:
train.head()

In [ ]:
# test["plot_synopsis"][2]

In [ ]:
vals = [i for i in df["tags"]]
print(len(vals))
vals = set(vals)
print(len(vals))

In [ ]:
import itertools
newl = [item.split(", ") for item in vals]
finl = list(itertools.chain.from_iterable(newl))
c=0
for item in finl:
    if not isinstance(item,str):
        c+=1
print(c)

In [ ]:
print(len(finl))
finl = set(finl)
print(len(finl))

In [ ]:
dict_map ={}
for index,tag in enumerate(finl):
    dict_map[tag] = index
print(dict_map)

In [ ]:
tokenizer(df["title"][0])

In [ ]:
row = {"tags":"horror, horror"}
print(dict_map[row["tags"].split(", ")[0]])

In [ ]:
def process_data(row):
    text = row["title"]
    encodings = tokenizer(text,padding="max_length",truncation=True,max_length=512)
    encodings["label"] = dict_map[row["tags"].split(", ")[0]]
    encodings["text"] = text
    return encodings 

In [ ]:
process_data({
'title':'Flightplan',
'tags':'mystery, suspenseful, action, murder, flashback'
    
})

In [ ]:
df.drop(["imdb_id","plot_synopsis","split","synopsis_source"],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
processed_data = []
for i in range(len(df)):
    processed_data.append(process_data(df.iloc[i]))


In [ ]:
from datasets import Dataset

In [ ]:
from sklearn.model_selection import train_test_split
new_df = pd.DataFrame(processed_data)
train_df,valid_df = train_test_split(new_df,test_size=0.2,random_state=2021)

In [ ]:
new_df.head()

In [ ]:
import pyarrow as pa
from datasets import Dataset

train_set = Dataset(pa.Table.from_pandas(train_df))
test_set = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
type(train_set)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=71)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./results",evaluation_strategy="epoch",num_train_epochs=5)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
)

In [ ]:
####ADD BATCHING BEFORE TRAINING TO REDUCE PARAMS
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model.save_pretrained("./model/")

In [ ]:
from transformers import AutoModelForSequenceClassification
new_model = AutoModelForSequenceClassification.from_pretrained("./model/")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
inverted_dict = dict(zip(dict_map.values(), dict_map.keys()))
inverted_dict

In [ ]:
import torch
import numpy as np

def get_prediction(text):
  encoding =tokenizer(text,return_tensors="pt",padding="max_length",truncation=True,max_length=512)
  encoding = {k: v.to("cpu") for k,v in encoding.items()}

  outputs = new_model(**encoding)
  logits = outputs.logits
  softmax = torch.nn.Softmax()
  probs = softmax(logits.squeeze().cpu())
  probs = probs.detach().numpy()
  top_3_indices = np.argpartition(probs, -3)[-3:]
  # top_3_values = probs[top_3_indices]

  # print(top_3_indices)
# print(top_3_values)
  top_3_values = [inverted_dict[item] for item in top_3_indices]
  # labels = np.argmax(probs,axis=-1)
  return {"tags":top_3_values}


In [ ]:
test.head()

In [ ]:
val.head()

In [ ]:
print(val["title"][20])

In [ ]:
get_prediction(val["title"][20])

In [ ]:
test.head()

In [ ]:
test.insert(loc=0, column='ID', value=np.arange(len(test)))


In [ ]:
test = test.reset_index()

In [ ]:
test.drop(labels=["index"],axis=1,inplace=True)

In [ ]:
test.head(40)

In [ ]:
test["plot_synopsis"][15]

In [ ]:
l = [i for i in test["ID"]]
# # print(l)
for i in l[:40]:
    print(f'{test["title"][i]} ',get_prediction(test["title"][i]))

In [ ]:
print(dict_map)